In [ ]:
# ===== Cell 1: 基本安裝 ======
!pip install -q docling rank-bm25 sentence-transformers faiss-cpu pandas

import os
from pathlib import Path
import pandas as pd

# 直接用左邊的視窗上傳檔案
DATA_DIR = Path("/content")

metadata = pd.read_csv(DATA_DIR / "metadata.csv", encoding="latin-1")
train_qa = pd.read_csv(DATA_DIR / "train_QA.csv", encoding="latin-1")
test_q   = pd.read_csv(DATA_DIR / "test_Q.csv",  encoding="latin-1")

print("metadata:", metadata.shape)
print("train_qa:", train_qa.shape)
print("test_q:",   test_q.shape)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.7/190.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 15.7 MB/s eta 0:00:00
   ━━━

In [ ]:
!pip install -q pdfplumber pdf2image pytesseract
!apt-get -y install poppler-utils tesseract-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 133.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (200 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-

In [ ]:
!pip install -q easyocr pillow opencv-python pdf2image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 20.7 MB/s eta 0:00:00


In [ ]:
# ====== Cell 2: 下載 PDF 到 /content/papers ======
import os
import requests
from pdfminer.high_level import extract_text
import logging, warnings

logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

PDF_DIR = Path("/content/papers")
PDF_DIR.mkdir(parents=True, exist_ok=True)

def download_pdf(url, doc_id, papers_dir=PDF_DIR):
    """
    下載單一 PDF 檔案：
    - filename = {doc_id}.pdf
    - 確認回傳內容真的是 PDF，避免下載 HTML
    """
    pdf_path = papers_dir / f"{doc_id}.pdf"

    if pdf_path.exists():
        return pdf_path

    print(f"⬇️ Downloading {doc_id} ...")
    try:
        resp = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
        if resp.status_code == 200 and resp.content[:4] == b"%PDF":
            with open(pdf_path, "wb") as f:
                f.write(resp.content)
            print(f"✅ Saved {pdf_path}")
            return pdf_path
        else:
            print(f"⚠️ Failed {doc_id}: status {resp.status_code}")
    except Exception as e:
        print(f"⚠️ Failed {doc_id}: {type(e).__name__}: {e}")
    return None

# 小小修正 metadata 的錯誤 URL
metadata.loc[metadata['id'] == 'zschache2025', 'url'] = 'https://arxiv.org/pdf/2508.14170'

success, fail, failed_ids = 0, 0, []

for _, row in metadata.iterrows():
    doc_id = row["id"]
    url    = row["url"]

    pdf_path = download_pdf(url, doc_id)
    if pdf_path is None:
        fail += 1
        failed_ids.append(doc_id)
    else:
        success += 1

print("="*60)
print(f"PDF 下載完成: 成功 {success}, 失敗 {fail}")
if failed_ids:
    print("失敗清單:", failed_ids)


⬇️ Downloading amazon2023 ...
✅ Saved /content/papers/amazon2023.pdf
⬇️ Downloading chen2024 ...
✅ Saved /content/papers/chen2024.pdf
⬇️ Downloading chung2025 ...
✅ Saved /content/papers/chung2025.pdf
⬇️ Downloading cottier2024 ...
✅ Saved /content/papers/cottier2024.pdf
⬇️ Downloading dodge2022 ...
✅ Saved /content/papers/dodge2022.pdf
⬇️ Downloading ebert2024 ...
✅ Saved /content/papers/ebert2024.pdf
⬇️ Downloading erben2023 ...
✅ Saved /content/papers/erben2023.pdf
⬇️ Downloading fernandez2025 ...
✅ Saved /content/papers/fernandez2025.pdf
⬇️ Downloading griggs2024 ...
✅ Saved /content/papers/griggs2024.pdf
⬇️ Downloading han2024 ...
✅ Saved /content/papers/han2024.pdf
⬇️ Downloading jegham2025 ...
✅ Saved /content/papers/jegham2025.pdf
⬇️ Downloading khan2025 ...
✅ Saved /content/papers/khan2025.pdf
⬇️ Downloading kim2025 ...
✅ Saved /content/papers/kim2025.pdf
⬇️ Downloading li2025a ...
✅ Saved /content/papers/li2025a.pdf
⬇️ Downloading li2025b ...
✅ Saved /content/papers/li2025b.p

In [ ]:
# ====== Cell 2: 初始化 EasyOCR (新增) ======
import easyocr
import cv2

print("🔧 初始化 EasyOCR...")
# 檢查是否有 GPU，沒有則用 CPU
try:
    import torch
    gpu_available = torch.cuda.is_available()
except:
    gpu_available = False

print(f"   GPU 可用: {gpu_available}")

# 初始化讀取器 (第一次會下載模型，約 100MB)
ocr_reader = easyocr.Reader(['en'], gpu=gpu_available)
print("✅ EasyOCR 初始化完成")

🔧 初始化 EasyOCR...
   GPU 可用: True
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ EasyOCR 初始化完成


In [ ]:
# ====== Cell 3: Docling 結構化拆解 + 圖片 OCR ======
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.document import TableItem, PictureItem, TextItem
from tqdm.auto import tqdm
import json
from PIL import Image
import io

CHUNK_DIR = Path("/content/wattbot_working/chunks")
IMAGE_DIR = Path("/content/wattbot_working/images")  # 新增：儲存提取的圖片

CHUNK_DIR.mkdir(parents=True, exist_ok=True)
IMAGE_DIR.mkdir(parents=True, exist_ok=True)

print("🚀 初始化 Docling Converter...")
converter = DocumentConverter(allowed_formats=[InputFormat.PDF])

def preprocess_image_for_ocr(image):
    """
    對圖片進行前處理，提高 OCR 準確度
    """
    if not isinstance(image, (Image.Image, type(None))):
        return image

    if image is None:
        return None

    try:
        # 轉為 OpenCV 格式
        img_cv = cv2.cvtColor(cv2.imread(image) if isinstance(image, str) else
                              cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR),
                              cv2.COLOR_BGR2GRAY) if not isinstance(image, np.ndarray) else image

        # 如果圖片太小，放大
        height, width = img_cv.shape[:2]
        if width < 400:
            scale = 400 / width
            img_cv = cv2.resize(img_cv, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)

        return img_cv
    except:
        return image

def extract_ocr_from_image(image, doc_id: str, page_no: int, img_index: int):
    """
    用 EasyOCR 從圖片提取文字

    Returns: (ocr_text, image_path) 或 (None, None)
    """
    try:
        # 如果是 PIL Image，轉成 numpy array
        if isinstance(image, Image.Image):
            img_array = np.array(image)
        elif isinstance(image, np.ndarray):
            img_array = image
        else:
            return None, None

        # 轉成 BGR 格式 (OpenCV 用)
        if len(img_array.shape) == 2:  # 灰度圖
            img_bgr = cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
        else:
            img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        # 保存圖片到磁碟（用於日後參考）
        image_filename = f"{doc_id}_p{page_no}_img{img_index}.png"
        image_path = IMAGE_DIR / image_filename
        cv2.imwrite(str(image_path), img_bgr)

        # 進行 OCR
        results = ocr_reader.readtext(img_bgr)

        # 提取文字並組合
        texts = []
        for (bbox, text, confidence) in results:
            if confidence > 0.2:  # 只要信心度 > 0.2 的文字
                texts.append(text)

        ocr_text = "\n".join(texts)

        if ocr_text.strip():
            return ocr_text, str(image_path)
        else:
            return None, None

    except Exception as e:
        print(f"  ❌ OCR 失敗: {e}")
        return None, None

def process_paper_structure(pdf_path: Path, doc_id: str):
    """
    用 Docling 把 PDF 轉成一系列 chunks：
    - type: "text" / "table" / "figure" / "image_ocr" (新增)
    - content: 給 BM25 / RAG 用的文字
    """
    chunks = []
    image_counter = 0

    try:
        result = converter.convert(str(pdf_path))
        doc    = result.document

        for item, level in doc.iterate_items():
            page_no = item.prov[0].page_no if item.prov else 0

            if isinstance(item, TableItem):
                table_md = item.export_to_markdown()
                content  = f"Table found on page {page_no}:\n{table_md}"
                chunks.append({
                    "doc_id": doc_id,
                    "type": "table",
                    "content": content,
                    "metadata": {"page": page_no},
                    "word_count": len(content.split()),
                })

            # ===== 新增：處理圖片 =====
            elif isinstance(item, PictureItem):
                # 先儲存圖片的基本資訊
                image_placeholder_content = f"[Figure on page {page_no}]"

                # 嘗試提取圖片並做 OCR
                ocr_text = None
                image_path = None

                try:
                    # 檢查 PictureItem 是否有圖片
                    if hasattr(item, 'image') and item.image is not None:
                        ocr_text, image_path = extract_ocr_from_image(
                            item.image,
                            doc_id,
                            page_no,
                            image_counter
                        )
                        image_counter += 1
                except Exception as e:
                    print(f"  ⚠️ 無法提取圖片 {doc_id} 第 {page_no} 頁: {e}")

                # 如果成功提取 OCR 文字，建立 image_ocr chunk
                if ocr_text:
                    content = f"[Image/Chart on page {page_no}]\nOCR Text:\n{ocr_text}"
                    chunks.append({
                        "doc_id": doc_id,
                        "type": "image_ocr",
                        "content": content,
                        "metadata": {
                            "page": page_no,
                            "image_path": image_path
                        },
                        "word_count": len(ocr_text.split()),
                    })
                else:
                    # 如果 OCR 失敗，仍然建立基本的 figure chunk
                    chunks.append({
                        "doc_id": doc_id,
                        "type": "figure",
                        "content": image_placeholder_content,
                        "metadata": {"page": page_no},
                        "word_count": len(image_placeholder_content.split()),
                    })

            elif isinstance(item, TextItem):
                text = item.text.strip()
                if len(text) < 20:
                    continue
                if text.lower() in ["references", "bibliography"]:
                    continue

                chunks.append({
                    "doc_id": doc_id,
                    "type": "text",
                    "content": text,
                    "metadata": {"page": page_no},
                    "word_count": len(text.split()),
                })

    except Exception as e:
        print(f"❌ Docling 處理失敗 {doc_id}: {e}")

    return chunks

# 處理所有 PDF
print("\n📄 開始處理 PDF...")
all_chunks = []
pdf_files  = list(PDF_DIR.glob("*.pdf"))

if not pdf_files:
    print("⚠️ 找不到任何 PDF，請先確認 Cell 1 是否下載成功")
else:
    for pdf_file in tqdm(pdf_files, desc="Processing PDFs with OCR"):
        doc_id = pdf_file.stem
        all_chunks.extend(process_paper_structure(pdf_file, doc_id))

# 儲存 chunks
out_file = CHUNK_DIR / "structured_chunks_with_ocr.jsonl"
with open(out_file, "w", encoding="utf-8") as f:
    for c in all_chunks:
        f.write(json.dumps(c, ensure_ascii=False) + "\n")

print("="*60)
print(f"✅ 處理完成")
print(f"   總 chunks 數: {len(all_chunks)}")
print(f"   儲存位置: {out_file}")
print(f"   圖片位置: {IMAGE_DIR}")

# 統計類型分布
from collections import Counter
type_counts = Counter(c["type"] for c in all_chunks)
print("\n📊 類型分布:")
for t, count in sorted(type_counts.items()):
    print(f"   {t}: {count}")

🚀 初始化 Docling Converter...

📄 開始處理 PDF...


Processing PDFs with OCR:   0%|          | 0/32 [00:00<?, ?it/s]

✅ 處理完成
   總 chunks 數: 8407
   儲存位置: /content/wattbot_working/chunks/structured_chunks_with_ocr.jsonl
   圖片位置: /content/wattbot_working/images

📊 類型分布:
   figure: 437
   table: 153
   text: 7817


In [ ]:
# ====== Cell 4: 建立檢索索引 (更新為使用含 OCR 的 chunks) ======
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import json
from collections import defaultdict
from tqdm.auto import tqdm

# 使用新的 chunks 檔案（含 OCR）
CHUNKS_FILE = CHUNK_DIR / "structured_chunks_with_ocr.jsonl"

# 載入 chunks
all_chunks = []
doc_ids_set = set()

with open(CHUNKS_FILE, "r", encoding="utf-8") as f:
    for line in f:
        c = json.loads(line)
        if "content" not in c:
            continue
        all_chunks.append(c)
        doc_ids_set.add(c["doc_id"])

print(f"載入 chunks: {len(all_chunks)}，文件數: {len(doc_ids_set)}")

# enhanced_tokenize
def enhanced_tokenize(text):
    text = str(text).lower()
    # 保留數字(含小數)、年份、單詞
    numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', text)
    years = re.findall(r'\b20\d{2}\b', text)
    words = [w for w in re.findall(r'\w+', text) if len(w) > 2]
    return numbers + years + words

# BM25
print("📊 建立 BM25 索引...")
bm25_corpus = [c['content'] for c in all_chunks]
tokenized_corpus = [enhanced_tokenize(doc) for doc in tqdm(bm25_corpus, desc="BM25 Tokenizing")]
bm25_index = BM25Okapi(tokenized_corpus)

# Dense (Vector)
print("🧠 正在編碼向量...")
model_name = 'BAAI/bge-large-en-v1.5'
try:
    embedding_model = SentenceTransformer(model_name)
except:
    print("⚠️ BGE-large 下載失敗或記憶體不足，降級使用 MiniLM")
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    embedding_model = SentenceTransformer(model_name)

texts_to_encode = [f"Type: {c.get('type', 'text').upper()}\n{c['content'][:1000]}" for c in all_chunks]
embeddings = embedding_model.encode(
    texts_to_encode,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

dimension = embeddings.shape[1]
dense_index = faiss.IndexFlatIP(dimension)
dense_index.add(embeddings)
print("✅ 索引構建完成")

# ====== 新增: 測試 OCR 效果 ======
print("\n" + "="*60)
print("📋 OCR 結果統計:")
ocr_chunks = [c for c in all_chunks if c['type'] == 'image_ocr']
if ocr_chunks:
    print(f"   成功提取 OCR 的圖片: {len(ocr_chunks)} 張")
    print(f"   平均字符數: {np.mean([c['word_count'] for c in ocr_chunks]):.0f}")
    print(f"   最多字符: {max([c['word_count'] for c in ocr_chunks])}")

    print("\n   🎨 OCR 樣本 (前 3 張):")
    for i, chunk in enumerate(ocr_chunks[:3]):
        preview = chunk['content'][:150].replace('\n', ' ')
        print(f"   {i+1}. [{chunk['doc_id']}] {preview}...")
else:
    print("   ⚠️ 未成功提取任何 OCR 文字")

載入 chunks: 8407，文件數: 32
📊 建立 BM25 索引...


BM25 Tokenizing:   0%|          | 0/8407 [00:00<?, ?it/s]

🧠 正在編碼向量...


Batches:   0%|          | 0/526 [00:00<?, ?it/s]

✅ 索引構建完成

📋 OCR 結果統計:
   ⚠️ 未成功提取任何 OCR 文字


In [ ]:
# ====== Cell 5: UNIT_KEYWORDS + extract_metadata_hints + search_rrf ======

# 單位擴展字典 (The "Cheat Code") ——
UNIT_KEYWORDS = {
    # 碳排相關
    "tCO2e": "carbon emissions co2 greenhouse footprint tonnes",
    "tCO2": "carbon emissions co2 greenhouse footprint tonnes",
    "kgCO2e": "carbon emissions co2 greenhouse footprint kilograms",
    "gCO2": "carbon intensity gram emissions",
    "gCO2e/kWh": "carbon intensity per kwh emissions electricity",

    # 能源單位
    "MWh": "energy electricity consumption power megawatt hour",
    "kWh": "energy electricity consumption power kilowatt hour",
    "Wh": "energy electricity consumption power watt hour",
    "TWh": "energy electricity consumption power terawatt hour",

    # 效率指標
    "PUE": "power usage effectiveness data center efficiency",
    "L/kWh": "water consumption per kwh cooling efficiency",

    # 水量
    "liters": "water consumption withdrawal usage volume",
    "gallons": "water consumption withdrawal usage volume",

    # 金額
    "USD": "cost price budget dollar investment money",
    "EUR": "euro cost price budget investment",

    # 模型規模 / token
    "parameters": "model size scale billion parameters",
    "tokens": "training tokens dataset size sequence length samples",

    # 時間
    "seconds": "time duration latency",
    "ms": "time duration latency milliseconds",
    "minutes": "time duration minutes",
    "hours": "time duration hours",
    "days": "time duration days",
    "years": "time duration age",

    # 硬體
    "GPUs": "gpu hardware accelerators nvidia cards",
    "A100_80GB_GPU": "nvidia a100 80gb gpu hardware accelerator",
    "H100 GPUs": "nvidia h100 gpu hardware accelerator",

    # 百分比 / 倍數
    "percent": "percentage rate ratio proportion share",
    "fold": "increase improvement speedup times",

    # 其他
    "samples": "batch size number of samples",
}


def extract_metadata_hints(query):
    """從問題中提取潛在的文檔ID暗示 (作者名/年份)"""
    query_lower = query.lower()
    hints = []

    # 1. 提取年份 (e.g., "In 2023...")
    years = re.findall(r'\b(20\d{2})\b', query)
    for year in years:
        hints.append(year)

    # 2. 提取作者名 (簡易版，比對 doc_id)
    # 如果 doc_id 是 "chen2024"，而問題有 "Chen"，就加分
    for doc_id in doc_ids_set:
        # 移除數字取作者名 (chen2024 -> chen)
        author_part = re.sub(r'\d+', '', doc_id).lower()
        if len(author_part) > 3 and author_part in query_lower:
            hints.append(doc_id)  # 直接把整個 doc_id 加入暗示

    return hints

def search_rrf(query, answer_unit="is_blank", top_k=10):
    """
    Full Recall 版本的檢索函數
    策略: BM25 + Vector + Unit擴展 + 表格加權 + 元數據鎖定
    """
    # --- A. 查詢預處理 ---
    # 1. 單位擴展（只影響 BM25，那段一樣）
    expanded_query = query
    unit_str = str(answer_unit).strip()
    if unit_str in UNIT_KEYWORDS:
        expanded_query = f"{query} {UNIT_KEYWORDS[unit_str]}"

    # 2. 元數據暗示提取
    meta_hints = extract_metadata_hints(query)

    # --- B. 執行檢索 (擴大召回至 150) ---
    initial_k = 150

    # BM25 用 expanded_query（含單位關鍵字）
    bm25_tokens = enhanced_tokenize(expanded_query)
    bm25_scores = bm25_index.get_scores(bm25_tokens)
    top_n_bm25 = np.argsort(bm25_scores)[::-1][:initial_k]

    # Dense：用原始 query（不加單位字串）——和你原本一樣
    query_embedding = embedding_model.encode([query], normalize_embeddings=True)
    dense_scores, dense_indices = dense_index.search(query_embedding, initial_k)
    dense_indices = dense_indices[0]

    # --- C. RRF 融合與加權 ---
    doc_scores = defaultdict(float)
    k = 60

    # 處理 BM25 結果
    for rank, idx in enumerate(top_n_bm25):
        chunk = all_chunks[idx]
        doc_id = chunk['doc_id']
        type_ = chunk.get('type', 'text')

        # 權重計算
        weight = 1.0
        if type_ == 'table':
         weight *= 2.0      # 改: 1.5 → 2.0
        if type_ == 'image_ocr':  # 新增
          weight *= 1.8      # 圖片 OCR 1.8 倍
        if any(hint in doc_id for hint in meta_hints):
          weight *= 2.0

        doc_scores[idx] += (1 / (k + rank + 1)) * weight

    # 處理 Dense 結果
    for rank, idx in enumerate(dense_indices):
        if idx == -1:
            continue
        chunk = all_chunks[idx]
        doc_id = chunk['doc_id']
        type_ = chunk.get('type', 'text')

        # 權重計算 (同上)
        weight = 1.0

        if type_ == 'table':
         weight *= 2.0      # 改: 1.5 → 2.0
        if type_ == 'image_ocr':  # 新增
          weight *= 1.8      # 圖片 OCR 1.8 倍
        if any(hint in doc_id for hint in meta_hints):
          weight *= 2.0

        doc_scores[idx] += (1 / (k + rank + 1)) * weight

    # --- D. 排序輸出 ---
    sorted_indices = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)[:top_k]

    results = []
    for idx, score in sorted_indices:
        chunk = all_chunks[idx]
        results.append({
            "doc_id": chunk['doc_id'],
            "content": chunk['content'],
            "type": chunk.get('type', 'text'),
            "score": score,
            "metadata": chunk.get('metadata', {})
        })

    return results


In [ ]:
# ==== Cell 6: CrossEncoder Reranker ====

from sentence_transformers import CrossEncoder
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("使用設備:", device)

try:
    cross_encoder = CrossEncoder(
        "BAAI/bge-reranker-v2-m3",
        device=device,
        max_length=1024
    )
    print("成功載入 BGE CrossEncoder（大型）")
except:
    print("⚠️載入大型 BGE 失敗，改用 MiniLM（小模型）")
    cross_encoder = CrossEncoder(
        "cross-encoder/ms-marco-MiniLM-L-12-v2",
        device=device,
        max_length=512
    )


使用設備: cuda
成功載入 BGE CrossEncoder（大型）


In [ ]:
# ==== Cell 7: rerank_results & search_with_reranking ====

def rerank_results(query, answer_unit, candidates, top_k=5):
    """
    對 RRF 召回結果進行 CrossEncoder 重排
    """
    augmented_query = f"{query} (Target unit: {answer_unit})"

    pairs = [
        [augmented_query, c["content"][:2000]]
        for c in candidates
    ]

    # CrossEncoder 打分
    ce_scores = cross_encoder.predict(pairs)

    for i, c in enumerate(candidates):
        c["ce_score"] = float(ce_scores[i])

    # 依照 ce_score 排序
    reranked = sorted(
        candidates,
        key=lambda x: x["ce_score"],
        reverse=True
    )[:top_k]

    # 標上 final_rank
    for rank, item in enumerate(reranked, start=1):
        item["final_rank"] = rank

    return reranked


def search_with_reranking(query, answer_unit="is_blank",
                          initial_k=100, final_k=5):
    """
    完整 pipeline：先 RRF → 再 CrossEncoder Rerank（保留組員原邏輯）
    """
    candidates = search_rrf(
        query,
        answer_unit,
        top_k=initial_k
    )

    reranked = rerank_results(
        query, answer_unit,
        candidates,
        top_k=final_k
    )

    return reranked


In [ ]:
# ==== Cell 8: should_use_reranking + detect_unanswerable ====

def should_use_reranking_query_based(query, results):
    """
    根據 Query 判斷是否啟用 Reranking
    """
    q = query.lower()
    keywords = ["compare", "percent", "ratio", "increase", "parameter",
                "emission", "mwh", "kwh", "co2", "usd", "table"]

    if any(k in q for k in keywords):
        return {"should_rerank": True, "confidence": "high"}

    # 一般題型 → 預設開啟
    return {"should_rerank": True, "confidence": "medium"}


def detect_unanswerable_bge(query, top_results):
    """
    只擋明顯不在語料裡的怪題，
    其他一律交給 LLM 嘗試回答（避免過度 is_blank）
    """
    q = query.lower()

    # 1) 明顯怪題：用關鍵字直接擋掉
    odd_keywords = [
        "earth from the sun",
        "distance from the sun",
        "elephant",
        "jupiter",
        "galaxy",
    ]
    if any(k in q for k in odd_keywords):
        return {"is_unanswerable": True, "reason": "weird query"}

    # 2) 如果真的一個檢索結果都沒有
    if not top_results:
        return {"is_unanswerable": True, "reason": "no retrieval results"}

    # 其他情況一律視為可回答
    return {"is_unanswerable": False}



In [ ]:
# ==== Cell 9: adaptive_search_final ====

def adaptive_search_final(query, answer_unit="is_blank",
                          initial_k=100, final_k=8):

    # 先做初步 RRF
    bm25_results = search_rrf(query, answer_unit, top_k=initial_k)

    # 決定要不要 rerank
    decision = should_use_reranking_query_based(query, bm25_results)

    if decision["should_rerank"]:
        final_results = rerank_results(
            query, answer_unit, bm25_results, top_k=final_k
        )
        strategy = f"Hybrid+BGE (confidence={decision['confidence']})"
    else:
        final_results = bm25_results[:final_k]
        strategy = "BM25-only"

    unanswerable = detect_unanswerable_bge(query, final_results)

    return {
        "results": final_results,
        "strategy": strategy,
        "used_reranking": decision["should_rerank"],
        "unanswerable": unanswerable
    }


In [ ]:
# ==== Cell 10: evaluate_with_details ====

import ast
import numpy as np
from collections import defaultdict

def evaluate_with_details(train_qa, search_func):
    recalls = {1:0, 3:0, 5:0, 10:0}
    mrr_total = 0
    total = 0
    failure_cases = []
    used_rerank_count = 0

    for _, row in train_qa.iterrows():
        q = row["question"]
        unit = row["answer_unit"]
        true_ids = row["ref_id"]

        try:
            true_docs = ast.literal_eval(true_ids)
        except:
            true_docs = [true_ids]

        res = search_func(q, answer_unit=unit, initial_k=150, final_k=10)
        retrieved = [r["doc_id"] for r in res["results"]]

        total += 1

        # 計算 Recall@K
        for k in [1,3,5,10]:
            top_k_set = set(retrieved[:k])
            if any(t in top_k_set for t in true_docs):
                recalls[k] += 1

        # MRR
        rr = 0
        for rank, did in enumerate(retrieved, start=1):
            if did in true_docs:
                rr = 1.0 / rank
                break
        mrr_total += rr

        # Reranking 使用率
        if res["used_reranking"]:
            used_rerank_count += 1

        # failure case
        if rr == 0:
            failure_cases.append({
                "question": q,
                "answer_unit": unit,
                "true_docs": true_docs,
                "retrieved": retrieved[:5],
                "strategy": res["strategy"]
            })

    return {
        "recall@1": recalls[1] / total,
        "recall@3": recalls[3] / total,
        "recall@5": recalls[5] / total,
        "recall@10": recalls[10] / total,
        "MRR": mrr_total / total,
        "total_questions": total,
        "failure_cases": failure_cases[:10],
        "rerank_usage": used_rerank_count / total
    }


# Gemini

In [ ]:
import pandas as pd
from collections import Counter

print("=" * 70)
print("🔍 Level 1: 快速診斷")
print("=" * 70)

# === 基本統計 ===
print("\n1️⃣ Chunks 基本統計:")
print(f"   總 chunks 數: {len(all_chunks)}")
print(f"   總文件數: {len(doc_ids_set)}")
print(f"   平均 chunk 大小: {np.mean([c['word_count'] for c in all_chunks]):.0f} 字")
print(f"   最小 chunk: {min([c['word_count'] for c in all_chunks])} 字")
print(f"   最大 chunk: {max([c['word_count'] for c in all_chunks])} 字")

# === 類型分布 ===
print("\n2️⃣ Chunk 類型分布:")
type_counts = Counter(c["type"] for c in all_chunks)
for t, count in sorted(type_counts.items()):
    pct = 100 * count / len(all_chunks)
    print(f"   {t:15s}: {count:5d} ({pct:5.1f}%)")

# === 文件分布 ===
print("\n3️⃣ 文件分布 (每份文件有多少 chunks):")
doc_chunk_counts = Counter(c["doc_id"] for c in all_chunks)
print(f"   最多 chunks: {max(doc_chunk_counts.values())} ({[k for k,v in doc_chunk_counts.items() if v == max(doc_chunk_counts.values())][0]})")
print(f"   最少 chunks: {min(doc_chunk_counts.values())}")
print(f"   平均 chunks/文件: {np.mean(list(doc_chunk_counts.values())):.1f}")

# === OCR 質量 ===
ocr_chunks = [c for c in all_chunks if c['type'] == 'image_ocr']
if ocr_chunks:
    print("\n4️⃣ OCR 提取質量:")
    print(f"   提取的 OCR chunks: {len(ocr_chunks)}")
    print(f"   OCR 平均字數: {np.mean([c['word_count'] for c in ocr_chunks]):.0f}")
    print(f"   OCR 最短: {min([c['word_count'] for c in ocr_chunks])} 字")
    print(f"   OCR 最長: {max([c['word_count'] for c in ocr_chunks])} 字")
else:
    print("\n4️⃣ ⚠️ 未找到任何 OCR chunks")

🔍 Level 1: 快速診斷

1️⃣ Chunks 基本統計:
   總 chunks 數: 8407
   總文件數: 32
   平均 chunk 大小: 46 字
   最小 chunk: 1 字
   最大 chunk: 959 字

2️⃣ Chunk 類型分布:
   figure         :   437 (  5.2%)
   table          :   153 (  1.8%)
   text           :  7817 ( 93.0%)

3️⃣ 文件分布 (每份文件有多少 chunks):
   最多 chunks: 1739 (amazon2023)
   最少 chunks: 78
   平均 chunks/文件: 262.7

4️⃣ ⚠️ 未找到任何 OCR chunks


In [ ]:
!pip install -q docling rank-bm25 sentence-transformers FlagEmbedding google-generativeai
!pip install -q pdfplumber easyocr nltk spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.5 MB/s eta 0:00:00


In [ ]:
# ==============================================================================
# 完整替換版本 - 保留所有 Prompt 細節
# 替換原本的 Cell 13 和 Cell 14
# ==============================================================================

# ==== 新 Cell 13: 完整 Gemini Generator（含所有 Prompt） ====

from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple

def detect_question_type(question: str) -> Tuple[str, bool]:
    """檢測問題類型，返回 (問題類型, 是否需要激進策略)"""
    q_lower = question.lower()

    # 類型 1: 表格數值提取
    table_extraction_patterns = [
        r'execution time', r'total time', r'throughput.*queries',
        r'queries.*per.*sec', r'batch size', r'maximum batch',
        r'gpu hours', r'h100.*hours', r'a100.*hours',
        r'total.*parameters', r'number of parameters',
        r'number of experts', r'how many experts',
        r'energy consumption.*wh', r'power usage.*kwh',
        r'cost.*\$|cost.*usd', r'price per hour',
        r'energy.*mwh|energy.*kwh', r'what is the .* for .* model',
    ]
    for pattern in table_extraction_patterns:
        if re.search(pattern, q_lower):
            return ("table_extraction", True)

    # 類型 2: 特定模型/論文
    specific_model_patterns = [
        (r'jetmoe', "model_specific"), (r'blackmamba', "model_specific"),
        (r'mixtral', "model_specific"), (r'flm-101b', "model_specific"),
        (r'bloom\b', "model_specific"), (r'olmo', "model_specific"),
        (r'llama.?\d', "model_specific"),
    ]
    for pattern, qtype in specific_model_patterns:
        if re.search(pattern, q_lower):
            return (qtype, True)

    # 類型 3: 分數/排名
    score_patterns = [
        r'what score', r'average score', r'final.* score',
        r'benchmark.*score', r'accuracy', r'performance score',
        r'leaderboard', r'gsm8k', r'mt-bench', r'bleu',
    ]
    for pattern in score_patterns:
        if re.search(pattern, q_lower):
            return ("score_extraction", True)

    # 類型 4: 術語/名稱
    term_patterns = [
        r'what is the name of', r'what is the term',
        r'which (?:software|framework|tool|package|metric)',
        r'what (?:software|framework|tool|package|metric)',
        r'what.*called',
    ]
    for pattern in term_patterns:
        if re.search(pattern, q_lower):
            return ("term_extraction", True)

    # 類型 5: 數量/計數
    count_patterns = [
        r'how many', r'how much', r'what percentage',
        r'what fraction', r'what proportion', r'by what factor',
        r'what was the .* increase', r'what was the .* decrease',
    ]
    for pattern in count_patterns:
        if re.search(pattern, q_lower):
            return ("numeric_extraction", True)

    return ("general", False)


def detect_multi_doc_question(question: str) -> Tuple[bool, str]:
    """保守地偵測是否需要多篇論文"""
    question_lower = question.lower()

    cross_calc_patterns = [
        r'equivalent to .+ (household|american|person|life)',
        r'how many .+ equivalent to', r'compare.*to',
    ]
    for pattern in cross_calc_patterns:
        if re.search(pattern, question_lower):
            return True, "cross_doc_calculation"

    if 'and how does this compare' in question_lower:
        return True, "explicit_comparison"

    return False, "single_doc"


def get_enhanced_table_prompt() -> str:
    """返回增強版的表格解析指引"""
    return """
📊 ENHANCED TABLE PARSING RULES (CRITICAL):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. TABLE STRUCTURE:
   - Tables use "|" to separate columns
   - First row = HEADER, subsequent rows = DATA
   - Example:
     | Model | Batch Size | Time (s) | Cost ($) |
     |-------|------------|----------|----------|
     | Dense |     30     |   45.2   |   12.5   |
     | Sparse|     84     |   32.1   |    8.7   |

2. HOW TO READ:
   - STEP 1: Find ROW matching question criteria (model, config)
   - STEP 2: Find COLUMN containing the answer type
   - STEP 3: Extract VALUE at intersection
   - STEP 4: Include UNIT from column header

3. CRITICAL MATCHING:
   - "dense" vs "sparse" → DIFFERENT rows!
   - "BlackMamba" vs "Mixtral" → DIFFERENT models!
   - "A40" vs "A100" vs "H100" → DIFFERENT GPUs!
   - "batch size = 1" vs "batch size = 30" → DIFFERENT rows!

4. IF YOU SEE A TABLE:
   - Check ALL rows before saying "Unable to answer"
   - The answer IS there if the model/config is mentioned
   - Extract EXACT number with full precision
"""


class EnhancedTwoStageGPTGenerator:
    """增強版兩階段生成器 - Gemini API"""

    def __init__(self, model: str = "gemini-2.5-pro", client=None, rpm_limit: int = None):
        self.model = model
        self.client = client
        self.call_count = 0
        self.total_cost = 0.0
        self.stage1_success = 0
        self.stage2_retry = 0
        self.multi_doc_count = 0
        self.aggressive_count = 0
        self.no_text_count = 0

        # 模型配置
        MODEL_CONFIGS = {
            "gemini-2.5-flash": {"input_price": 0.075, "output_price": 0.30, "rpm_limit": 1000},
            "gemini-2.5-pro": {"input_price": 1.25, "output_price": 5.0, "rpm_limit": 150},
            "gemini-3-pro-preview": {"input_price": 2.0, "output_price": 12.0, "rpm_limit": 50},
            "gemini-2.0-flash": {"input_price": 0.10, "output_price": 0.40, "rpm_limit": 2000}
        }

        if model in MODEL_CONFIGS:
            self.model_config = MODEL_CONFIGS[model]
            self.rpm_limit = rpm_limit or self.model_config['rpm_limit']
            print(f"📌 使用模型: {model}")
            print(f"   定價: ${self.model_config['input_price']}/${self.model_config['output_price']} per 1M tokens")
            print(f"   Rate Limit: {self.rpm_limit}/min")
        else:
            self.model_config = MODEL_CONFIGS["gemini-2.5-flash"]
            self.rpm_limit = rpm_limit or 150

        self.call_timestamps = []

    def _wait_for_rate_limit(self):
        """確保不超過 rate limit"""
        now = datetime.now()
        self.call_timestamps = [ts for ts in self.call_timestamps if now - ts < timedelta(minutes=1)]

        if len(self.call_timestamps) >= self.rpm_limit:
            oldest_call = min(self.call_timestamps)
            wait_time = 60 - (now - oldest_call).total_seconds()
            if wait_time > 0:
                print(f"  ⏳ Rate limit, waiting {wait_time:.1f}s...")
                time.sleep(wait_time + 1)

        self.call_timestamps.append(datetime.now())

    def generate_answer(self, question: str, retrieval_results: List[Dict], expected_unit: str = "is_blank") -> Dict:
        """兩階段生成主函數"""

        # 問題類型檢測
        question_type, needs_aggressive = detect_question_type(question)
        if needs_aggressive:
            self.aggressive_count += 1

        # 多文檔檢測
        is_multi_doc, reason = detect_multi_doc_question(question)
        if is_multi_doc:
            self.multi_doc_count += 1

        # Stage 1: Top-3
        top3_results = retrieval_results[:3]
        context_top3 = self._build_context(top3_results)

        stage1_response = self._call_gpt(
            question=question, context=context_top3, retrieval_results=top3_results,
            expected_unit=expected_unit, mode="aggressive" if needs_aggressive else "strict",
            question_type=question_type, is_multi_doc=is_multi_doc
        )

        is_unable = "Unable to answer" in stage1_response.get('answer', '')

        if not is_unable:
            self.stage1_success += 1
            return stage1_response

        # Stage 2: Top-7 或 Top-10
        self.stage2_retry += 1
        top_k = 10 if needs_aggressive else 7
        top_results = retrieval_results[:top_k]
        context_expanded = self._build_context(top_results)

        stage2_response = self._call_gpt(
            question=question, context=context_expanded, retrieval_results=top_results,
            expected_unit=expected_unit, mode="aggressive", question_type=question_type,
            is_multi_doc=is_multi_doc
        )

        return stage2_response

    def _build_context(self, results: List[Dict]) -> str:
        """構建上下文 - 增強表格標記"""
        context_parts = []

        for i, r in enumerate(results, 1):
            content = r['content']
            chunk_type = r.get('type', 'text')
            doc_id = r['doc_id']

            if chunk_type == 'table':
                prefix = f"[DOCUMENT #{i}: {doc_id}]\n⚠️ DATA TABLE - READ CAREFULLY:\n{'='*60}\n"
            else:
                prefix = f"[DOCUMENT #{i}: {doc_id}]\n"

            context_parts.append(prefix + content)

        return "\n\n".join(context_parts)

    def _extract_text_from_response(self, response) -> Optional[str]:
        """從 Gemini 回應中提取文字"""
        try:
            try:
                text = response.text
            except Exception:
                text = None

            if isinstance(text, str) and text.strip():
                return text.strip()

            if hasattr(response, "parts") and response.parts:
                text_parts = []
                for part in response.parts:
                    part_text = getattr(part, "text", None)
                    if isinstance(part_text, str) and part_text.strip():
                        text_parts.append(part_text.strip())
                if text_parts:
                    return "\n".join(text_parts)

            if hasattr(response, "candidates") and response.candidates:
                for cand in response.candidates:
                    content = getattr(cand, "content", None)
                    if content is None:
                        continue
                    parts = getattr(content, "parts", None)
                    if not parts:
                        continue
                    text_parts = []
                    for part in parts:
                        part_text = getattr(part, "text", None)
                        if isinstance(part_text, str) and part_text.strip():
                            text_parts.append(part_text.strip())
                    if text_parts:
                        return "\n".join(text_parts)

            return None
        except Exception as e:
            print(f"  ⚠️ 提取文本異常: {e}")
            return None

    def _call_gpt(self, question, context, retrieval_results, expected_unit, mode, question_type, is_multi_doc):
        """調用 Gemini API"""

        if mode == "aggressive":
            system_prompt = self._get_aggressive_system_prompt(question_type, is_multi_doc)
        else:
            system_prompt = self._get_strict_system_prompt(is_multi_doc)

        user_prompt = self._build_user_prompt(question, context, expected_unit, retrieval_results, question_type, is_multi_doc)

        max_retries = 3
        retry_count = 0

        while retry_count < max_retries:
            try:
                self._wait_for_rate_limit()

                full_prompt = f"{system_prompt}\n\n---\n\n{user_prompt}"

                from google.genai import types
                safety_settings = [
                    types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="BLOCK_NONE"),
                    types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="BLOCK_NONE"),
                    types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="BLOCK_NONE"),
                    types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="BLOCK_NONE")
                ]

                gen_config = types.GenerateContentConfig(
                    temperature=0.1,
                    max_output_tokens=8192,
                    response_mime_type="application/json",
                    safety_settings=safety_settings
                )

                response = self.client.models.generate_content(
                    model=self.model, contents=full_prompt, config=gen_config,
                )

                self.call_count += 1
                content = self._extract_text_from_response(response)

                if content is None or content.strip() == '':
                    self.no_text_count += 1
                    reason = "UNKNOWN"
                    try:
                        if response.candidates:
                            reason = response.candidates[0].finish_reason
                    except:
                        pass

                    if str(reason) == "RECITATION":
                        return self._default_answer(expected_unit)

                    if retry_count < max_retries - 1:
                        retry_count += 1
                        time.sleep(3)
                        continue
                    else:
                        return self._default_answer(expected_unit)

                result = self._parse_response(content, expected_unit)

                try:
                    input_tokens = response.usage_metadata.prompt_token_count
                    output_tokens = response.usage_metadata.candidates_token_count
                    cost = (input_tokens * self.model_config['input_price'] + output_tokens * self.model_config['output_price']) / 1_000_000
                    self.total_cost += cost
                except:
                    self.total_cost += 0.001

                return result

            except Exception as e:
                error_str = str(e)
                if "429" in error_str or "RESOURCE_EXHAUSTED" in error_str:
                    retry_count += 1
                    if retry_count < max_retries:
                        wait_time = min(10 * (2 ** retry_count), 60)
                        time.sleep(wait_time)
                        continue

                if retry_count < max_retries - 1:
                    retry_count += 1
                    continue
                return self._default_answer(expected_unit)

        return self._default_answer(expected_unit)

    def _get_aggressive_system_prompt(self, question_type: str, is_multi_doc: bool) -> str:
        """激進模式 System Prompt"""

        table_section = get_enhanced_table_prompt()

        multi_doc_section = """
⭐ MULTI-DOCUMENT CITATION:
- Combine data from multiple documents if needed
- List all sources: ["doc1", "doc2"]
""" if is_multi_doc else ""

        type_hints = {
            "table_extraction": """
🎯 TABLE EXTRACTION MODE:
- The answer IS in a table - search ALL rows carefully!
- Match: model name + GPU type + batch size + dense/sparse
- DO NOT say "Unable to answer" if you see relevant tables!
""",
            "model_specific": """
🎯 MODEL-SPECIFIC MODE:
- Look for data specific to this MODEL
- Common data: parameter counts, expert counts, scores, GPU hours, tokens
- If model name appears in context, the answer should be there!
""",
            "score_extraction": """
🎯 SCORE EXTRACTION MODE:
- Look for benchmark tables or score mentions
- Common: GSM8k, MT-Bench, OpenLLM Leaderboard, accuracy, BLEU
- Extract the EXACT number with full precision
""",
            "term_extraction": """
🎯 TERM/NAME EXTRACTION MODE:
- Look for definitions, tool names, framework names, metrics
- The answer is a proper noun or technical term
- Search for quoted or emphasized text
""",
            "numeric_extraction": """
🎯 NUMERIC EXTRACTION MODE:
- Look for specific numbers, percentages, counts
- The answer exists in the documents
- Extract the exact value mentioned
""",
        }

        type_hint = type_hints.get(question_type, "")

        return f"""⚠️ JSON OUTPUT RULES:
- Output a single valid JSON object
- ALL fields: answer, answer_value, answer_unit, supporting_text, explanation, source_doc_ids
- Use "is_blank" for empty values (never null)
- source_doc_ids = LIST: ["doc_id"]

{type_hint}
{table_section}
{multi_doc_section}

⚠️ AGGRESSIVE MODE - REDUCED "UNABLE TO ANSWER" THRESHOLD:
- Only say "Unable to answer" if CERTAIN the info doesn't exist
- If you see ANY relevant data, extract it!
- Partial matches acceptable - use best available info
- For numerical questions, extract the closest matching value
- DO NOT refuse to answer if you see the model/topic mentioned!

🔢 UNIT HANDLING:
- MWh → kWh: ×1000
- kWh → Wh: ×1000
- million → raw: ×1,000,000
- billion → raw: ×1,000,000,000
- Always check if conversion is needed!

🧠 EXPLANATION:
- For table data: "Found in row X, column Y of table"
- Show calculations if needed
- Cite the specific text/number you found

You are a data extraction expert. PREFER ANSWERING over refusing."""

    def _get_strict_system_prompt(self, is_multi_doc: bool) -> str:
        """嚴格模式 System Prompt"""

        table_section = get_enhanced_table_prompt()

        multi_doc_section = """
⭐ MULTI-DOCUMENT CITATION:
- List sources as: ["doc1", "doc2"]
""" if is_multi_doc else ""

        return f"""⚠️ JSON OUTPUT RULES:
- Output a single valid JSON object
- ALL fields: answer, answer_value, answer_unit, supporting_text, explanation, source_doc_ids
- Use "is_blank" for empty values
- source_doc_ids = LIST

⚠️ UNABLE TO ANSWER:
- If cannot answer: answer = "Unable to answer with confidence based on the provided documents."
- answer_value = "is_blank", source_doc_ids = []
- DO NOT guess or fabricate
{multi_doc_section}
{table_section}

🔢 UNIT CONVERSION:
- MWh → kWh: ×1000
- million → raw: ×1,000,000

✅ STRICT MODE: Answer only if evidence is explicit and clear.

You are a scientific evidence extraction expert."""

    def _build_user_prompt(self, question, context, expected_unit, retrieval_results, question_type, is_multi_doc):
        """構建 User Prompt"""

        available_doc_ids = list(set([r['doc_id'] for r in retrieval_results]))
        doc_id_list = ", ".join(available_doc_ids)

        type_hints = {
            "table_extraction": "⚠️ HINT: Answer is in a TABLE. Search all rows carefully!",
            "model_specific": "⚠️ HINT: Look for this specific MODEL's data in the context.",
            "score_extraction": "⚠️ HINT: Look for benchmark scores or performance metrics.",
            "term_extraction": "⚠️ HINT: Look for technical terms, tool names, or definitions.",
            "numeric_extraction": "⚠️ HINT: Look for specific numbers or percentages.",
        }
        hint = type_hints.get(question_type, "")

        multi_hint = "\n⚠️ May need to combine data from multiple documents." if is_multi_doc else ""

        return f"""QUESTION:
{question}

{hint}{multi_hint}

EXPECTED UNIT: {expected_unit}

AVAILABLE DOC IDs: {doc_id_list}

CONTEXT:
{context[:12000]}

OUTPUT FORMAT (STRICT JSON):
{{
  "answer": "natural language answer with unit",
  "answer_value": "number or keyword (NO unit, NO symbols like <, >, ~)",
  "answer_unit": "{expected_unit}",
  "supporting_text": "verbatim quote from context (≤30 words)",
  "explanation": "step-by-step reasoning (mention row/col for tables)",
  "source_doc_ids": ["doc_id"]
}}

SPECIAL CASES:
- TRUE/FALSE: answer_value = "1" (TRUE) or "0" (FALSE)
- Range: answer_value = "[low,high]" (no spaces inside brackets)
- Unable to answer: ONLY if you searched ALL context and found NOTHING relevant

GENERATE JSON NOW:"""

    def _parse_response(self, content, expected_unit):
        """解析 Gemini 響應"""
        try:
            content = content.replace('```json', '').replace('```', '').strip()
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if not json_match:
                return self._default_answer(expected_unit)

            result = json.loads(json_match.group())

            required_fields = ['answer', 'answer_value', 'answer_unit', 'supporting_text', 'explanation', 'source_doc_ids']
            for field in required_fields:
                if field not in result or result[field] is None:
                    result[field] = [] if field == 'source_doc_ids' else 'is_blank'

            doc_ids = result.get('source_doc_ids', [])
            if isinstance(doc_ids, str):
                result['source_doc_ids'] = [] if doc_ids == 'is_blank' or not doc_ids else [doc_ids]
            elif not isinstance(doc_ids, list):
                result['source_doc_ids'] = []

            result['source_doc_ids'] = list(set(result['source_doc_ids']))

            answer_text = result.get('answer', '').lower()
            if 'unable to answer' in answer_text or 'cannot answer' in answer_text or 'not able to answer' in answer_text:
                return {
                    'answer': 'Unable to answer with confidence based on the provided documents.',
                    'answer_value': 'is_blank',
                    'answer_unit': expected_unit,
                    'supporting_text': 'is_blank',
                    'explanation': 'is_blank',
                    'source_doc_ids': []
                }

            answer_str = str(result.get('answer', '')).strip()
            answer_val = str(result.get('answer_value', '')).strip().upper()

            if answer_str.upper() in ['TRUE', 'FALSE'] or answer_val in ['TRUE', 'FALSE']:
                is_true = answer_str.upper() == 'TRUE' or answer_val == 'TRUE'
                result['answer'] = 'TRUE' if is_true else 'FALSE'
                result['answer_value'] = '1' if is_true else '0'
                result['answer_unit'] = 'is_blank'
            else:
                result['answer_unit'] = expected_unit

            for key in result:
                if result[key] is None or result[key] == 'null':
                    result[key] = [] if key == 'source_doc_ids' else 'is_blank'

            return result

        except Exception as e:
            print(f"  解析錯誤: {e}")
            return self._default_answer(expected_unit)

    def _default_answer(self, expected_unit):
        """默認答案"""
        return {
            'answer': "Unable to answer with confidence based on the provided documents.",
            'answer_value': 'is_blank',
            'answer_unit': expected_unit,
            'supporting_text': 'is_blank',
            'explanation': 'is_blank',
            'source_doc_ids': []
        }

    def print_stats(self):
        """打印統計"""
        print(f"\n📊 統計:")
        print(f"  模型: {self.model}")
        print(f"  總調用: {self.call_count}")
        print(f"  Stage 1 成功: {self.stage1_success}")
        print(f"  Stage 2 重試: {self.stage2_retry}")
        print(f"  激進模式: {self.aggressive_count}")
        print(f"  多文檔: {self.multi_doc_count}")
        print(f"  無文本: {self.no_text_count}")
        print(f"  總成本: ${self.total_cost:.4f} USD")
        if self.call_count > 0:
            print(f"  平均成本: ${self.total_cost/self.call_count:.4f} USD")


# ==== Cell 14: 生成 submission ====

# 初始化 Gemini client
from google import genai
GEMINI_API_KEY = "# ⚠️ 請替換成你的 API Key"
client = genai.Client(api_key=GEMINI_API_KEY)

# 讀取資料
test_q = pd.read_csv("/content/test_Q.csv", encoding="utf-8-sig")

# 初始化生成器
generator = EnhancedTwoStageGPTGenerator(model="gemini-2.5-pro", client=client)

# 生成答案
records = []

print(f"開始生成 {len(test_q)} 題答案...")

for _, row in tqdm(test_q.iterrows(), total=len(test_q), desc="生成中"):
    qid = row['id']
    question = row['question']
    unit = row.get('answer_unit', 'is_blank')
    if pd.isna(unit):
        unit = 'is_blank'

    # 🎯 只需要這兩個參數！
    out = adaptive_search_final(question, answer_unit=unit)

    # 生成答案
    if out["unanswerable"]["is_unanswerable"] or not out["results"]:
        res = {
            'id': qid, 'question': question,
            'answer': 'Unable to answer with confidence based on the provided documents.',
            'answer_value': 'is_blank', 'answer_unit': unit,
            'ref_id': 'is_blank', 'ref_url': 'is_blank',
            'supporting_materials': 'is_blank', 'explanation': 'is_blank'
        }
    else:
        ans = generator.generate_answer(question, out["results"], unit)

        # 處理 ref_id
        is_unable = 'Unable to answer' in ans.get('answer', '')
        if is_unable:
            ref_id = 'is_blank'
            ref_url = 'is_blank'
        else:
            doc_ids = ans.get('source_doc_ids', [])
            valid_ids = []
            valid_urls = []

            for did in doc_ids:
                if did and did != 'is_blank':
                    mask = metadata['id'] == did
                    if mask.any():
                        valid_ids.append(did)
                        valid_urls.append(metadata[mask]['url'].values[0])

            if not valid_ids and out["results"]:
                fallback = out["results"][0]['doc_id']
                mask = metadata['id'] == fallback
                if mask.any():
                    valid_ids = [fallback]
                    valid_urls = [metadata[mask]['url'].values[0]]

            ref_id = str(valid_ids).replace('"', "'") if valid_ids else 'is_blank'
            ref_url = str(valid_urls).replace('"', "'") if valid_urls else 'is_blank'

        res = {
            'id': qid, 'question': question,
            'answer': ans.get('answer', 'is_blank'),
            'answer_value': ans.get('answer_value', 'is_blank'),
            'answer_unit': unit,
            'ref_id': ref_id, 'ref_url': ref_url,
            'supporting_materials': ans.get('supporting_text', 'is_blank'),
            'explanation': ans.get('explanation', 'is_blank')
        }

    records.append(res)
    time.sleep(1.0)

# 保存
submission = pd.DataFrame(records)
submission = submission.fillna('is_blank')
submission.to_csv("/content/submission_enhanced.csv", index=False, encoding="utf-8-sig")

unable_count = sum(submission['answer'].str.contains('Unable to answer', na=False))
print(f"\n✅ 完成! Unable: {unable_count}/{len(submission)} 題")

generator.print_stats()

📌 使用模型: gemini-2.5-pro
   定價: $1.25/$5.0 per 1M tokens
   Rate Limit: 150/min
開始生成 282 題答案...


生成中:   0%|          | 0/282 [00:00<?, ?it/s]


✅ 完成! Unable: 26/282 題

📊 統計:
  模型: gemini-2.5-pro
  總調用: 323
  Stage 1 成功: 240
  Stage 2 重試: 41
  激進模式: 177
  多文檔: 10
  無文本: 1
  總成本: $0.9844 USD
  平均成本: $0.0030 USD


清除快取

In [ ]:
!rm /content/docling_cache.pkl
!rm /content/bm25_index.pkl
!rm /content/dense_index.pkl

rm: cannot remove '/content/docling_cache.pkl': No such file or directory
rm: cannot remove '/content/bm25_index.pkl': No such file or directory
rm: cannot remove '/content/dense_index.pkl': No such file or directory


In [ ]:
# ==== 評估指標計算與輸出 (修正版：處理重複文件) ====

import ast
import numpy as np
from collections import defaultdict

def calculate_ndcg_at_k(retrieved, relevant, k):
    """
    計算 nDCG@k (修正版：正確處理重複文件)

    Args:
        retrieved: 檢索結果的 doc_id 列表 (可能包含重複)
        relevant: 相關文件的 doc_id 列表
        k: 評估的 top-k

    Returns:
        nDCG@k 分數 (保證在 0-1 之間)
    """
    # DCG@k - 只計算每個相關文件的第一次出現
    dcg = 0.0
    seen_docs = set()  # 追蹤已經計分的文件

    for i, doc_id in enumerate(retrieved[:k]):
        # 只有在首次出現且是相關文件時才計分
        if doc_id in relevant and doc_id not in seen_docs:
            dcg += 1.0 / np.log2(i + 2)  # i+2 因為位置從 1 開始
            seen_docs.add(doc_id)

    # IDCG@k (理想情況：所有相關文件都在最前面且不重複)
    idcg = 0.0
    for i in range(min(len(relevant), k)):
        idcg += 1.0 / np.log2(i + 2)

    # nDCG
    if idcg == 0:
        return 0.0

    ndcg = dcg / idcg

    # 確保結果在 [0, 1] 範圍內 (理論上應該自動滿足，但加個檢查)
    assert 0 <= ndcg <= 1.0001, f"nDCG={ndcg} 超出範圍 [0,1]！DCG={dcg}, IDCG={idcg}"

    return min(ndcg, 1.0)  # 防止浮點數誤差導致略大於1


def evaluate_with_all_metrics(train_qa, search_func):
    """
    完整評估函數：計算 Recall@K, MRR, nDCG@K

    Args:
        train_qa: 包含問題和標準答案的 DataFrame
        search_func: 檢索函數，需要接受 (query, answer_unit, initial_k, final_k) 參數

    Returns:
        包含所有評估指標的字典
    """
    recalls = {1: 0, 3: 0, 5: 0, 10: 0}
    ndcgs = {1: 0, 3: 0, 5: 0, 10: 0}
    mrr_total = 0
    total = 0
    failure_cases = []
    used_rerank_count = 0
    duplicate_count = 0  # 追蹤有重複文件的查詢數量

    for _, row in train_qa.iterrows():
        q = row["question"]
        unit = row["answer_unit"]
        true_ids = row["ref_id"]

        # 解析標準答案的文件 ID
        try:
            true_docs = ast.literal_eval(true_ids)
        except:
            true_docs = [true_ids]

        # 確保 true_docs 是列表
        if not isinstance(true_docs, list):
            true_docs = [true_ids]

        # 執行檢索
        res = search_func(q, answer_unit=unit, initial_k=150, final_k=10)
        retrieved = [r["doc_id"] for r in res["results"]]

        # 檢查是否有重複
        if len(retrieved) != len(set(retrieved)):
            duplicate_count += 1

        total += 1

        # 計算 Recall@K
        for k in [1, 3, 5, 10]:
            top_k_set = set(retrieved[:k])
            if any(t in top_k_set for t in true_docs):
                recalls[k] += 1

        # 計算 nDCG@K (使用修正後的函數)
        for k in [1, 3, 5, 10]:
            ndcg_score = calculate_ndcg_at_k(retrieved, true_docs, k)
            ndcgs[k] += ndcg_score

        # 計算 MRR (Mean Reciprocal Rank)
        rr = 0
        for rank, did in enumerate(retrieved, start=1):
            if did in true_docs:
                rr = 1.0 / rank
                break
        mrr_total += rr

        # 記錄是否使用 reranking
        if res.get("used_reranking", False):
            used_rerank_count += 1

        # 記錄失敗案例 (MRR = 0 表示沒找到任何相關文件)
        if rr == 0:
            failure_cases.append({
                "question": q,
                "answer_unit": unit,
                "true_docs": true_docs,
                "retrieved": retrieved[:5],
                "strategy": res.get("strategy", "unknown")
            })

    # 計算平均值
    results = {
        "recall@1": recalls[1] / total,
        "recall@3": recalls[3] / total,
        "recall@5": recalls[5] / total,
        "recall@10": recalls[10] / total,
        "ndcg@1": ndcgs[1] / total,
        "ndcg@3": ndcgs[3] / total,
        "ndcg@5": ndcgs[5] / total,
        "ndcg@10": ndcgs[10] / total,
        "mrr": mrr_total / total,
        "total_questions": total,
        "duplicate_queries": duplicate_count,  # 新增：有重複的查詢數
        "duplicate_rate": duplicate_count / total if total > 0 else 0,
        "failure_cases": failure_cases[:10],
        "rerank_usage": used_rerank_count / total if total > 0 else 0
    }

    return results


def print_evaluation_results(results):
    """
    漂亮地印出評估結果

    Args:
        results: evaluate_with_all_metrics 的返回結果
    """
    print("\n" + "=" * 70)
    print("📊 檢索系統評估結果")
    print("=" * 70)

    print(f"\n📈 總題數: {results['total_questions']}")
    print(f"🔄 Reranking 使用率: {results['rerank_usage']:.2%}")

    # 顯示重複文件警告
    if results.get('duplicate_queries', 0) > 0:
        dup_rate = results['duplicate_rate']
        print(f"⚠️  重複文件警告: {results['duplicate_queries']}/{results['total_questions']} 個查詢 ({dup_rate:.1%}) 包含重複文件")

    print("\n" + "-" * 70)
    print("🎯 Recall@K (至少找到一個相關文件的比例)")
    print("-" * 70)
    print(f"  Recall@1:  {results['recall@1']:.4f} ({results['recall@1']:.2%})")
    print(f"  Recall@3:  {results['recall@3']:.4f} ({results['recall@3']:.2%})")
    print(f"  Recall@5:  {results['recall@5']:.4f} ({results['recall@5']:.2%})")
    print(f"  Recall@10: {results['recall@10']:.4f} ({results['recall@10']:.2%})")

    print("\n" + "-" * 70)
    print("📊 nDCG@K (考慮排序質量的評估指標)")
    print("-" * 70)
    print(f"  nDCG@1:  {results['ndcg@1']:.4f}")
    print(f"  nDCG@3:  {results['ndcg@3']:.4f}")
    print(f"  nDCG@5:  {results['ndcg@5']:.4f}")
    print(f"  nDCG@10: {results['ndcg@10']:.4f}")

    print("\n" + "-" * 70)
    print("🏆 MRR (Mean Reciprocal Rank)")
    print("-" * 70)
    print(f"  MRR: {results['mrr']:.4f}")
    if results['mrr'] > 0:
        print(f"  (平均第一個相關文件出現在第 {1/results['mrr']:.2f} 位)")

    # 失敗案例分析
    if results['failure_cases']:
        print("\n" + "-" * 70)
        print(f"❌ 失敗案例 (前 {len(results['failure_cases'])} 個)")
        print("-" * 70)
        for i, case in enumerate(results['failure_cases'], 1):
            print(f"\n案例 {i}:")
            print(f"  問題: {case['question'][:80]}...")
            print(f"  單位: {case['answer_unit']}")
            print(f"  正確文件: {case['true_docs']}")
            print(f"  檢索結果 (前5): {case['retrieved']}")
            print(f"  策略: {case['strategy']}")

    print("\n" + "=" * 70)


# ==== 使用範例 ====

if __name__ == "__main__":
    # 假設你已經有以下變數：
    # - train_qa: 包含問題和標準答案的 DataFrame
    # - adaptive_search_final: 你的檢索函數

    # 執行評估
    results = evaluate_with_all_metrics(train_qa, adaptive_search_final)

    # 印出結果
    print_evaluation_results(results)

    # 如果需要，也可以存成 CSV
    import pandas as pd
    metrics_df = pd.DataFrame([{
        'Recall@1': results['recall@1'],
        'Recall@3': results['recall@3'],
        'Recall@5': results['recall@5'],
        'Recall@10': results['recall@10'],
        'nDCG@1': results['ndcg@1'],
        'nDCG@3': results['ndcg@3'],
        'nDCG@5': results['ndcg@5'],
        'nDCG@10': results['ndcg@10'],
        'MRR': results['mrr'],
        'Rerank_Usage': results['rerank_usage'],
        'Duplicate_Rate': results.get('duplicate_rate', 0)
    }])

    metrics_df.to_csv('evaluation_metrics.csv', index=False)
    print("\n✅ 評估指標已儲存至 evaluation_metrics.csv")


📊 檢索系統評估結果

📈 總題數: 41
🔄 Reranking 使用率: 100.00%
⚠️  重複文件警告: 41/41 個查詢 (100.0%) 包含重複文件

----------------------------------------------------------------------
🎯 Recall@K (至少找到一個相關文件的比例)
----------------------------------------------------------------------
  Recall@1:  0.8049 (80.49%)
  Recall@3:  0.8780 (87.80%)
  Recall@5:  0.9268 (92.68%)
  Recall@10: 0.9512 (95.12%)

----------------------------------------------------------------------
📊 nDCG@K (考慮排序質量的評估指標)
----------------------------------------------------------------------
  nDCG@1:  0.8049
  nDCG@3:  0.8384
  nDCG@5:  0.8584
  nDCG@10: 0.8665

----------------------------------------------------------------------
🏆 MRR (Mean Reciprocal Rank)
----------------------------------------------------------------------
  MRR: 0.8519
  (平均第一個相關文件出現在第 1.17 位)

----------------------------------------------------------------------
❌ 失敗案例 (前 2 個)
----------------------------------------------------------------------

案例 1:
  問題: What was